## Importando Dataset - Campeonato Brasileiro

In [22]:
import pandas as pd
import numpy as np

csv_path = '.\dataset\campeonato-brasileiro-full.csv'

df = pd.read_csv(csv_path, delimiter=';')

## Capturando informações gerais das colunas

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8440 entries, 0 to 8439
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                8440 non-null   int64  
 1   Rodada            8440 non-null   object 
 2   Data              8436 non-null   object 
 3   Horário           8079 non-null   object 
 4   Dia               8436 non-null   object 
 5   Mandante          8440 non-null   object 
 6   Visitante         8440 non-null   object 
 7   Vencedor          8440 non-null   object 
 8   Arena             8436 non-null   object 
 9   Mandante Placar   8436 non-null   float64
 10  Visitante Placar  8436 non-null   float64
 11  Estado Mandante   8440 non-null   object 
 12  Estado Visitante  8440 non-null   object 
 13  Estado Vencedor   8440 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 923.2+ KB


## Listando os 5 primeiros registros

In [41]:
df.head(5)

,ID,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor,Pontos Mandante,Pontos Visitante,Diferenca Gols
0,1,1,2000-07-29,16h00,SABADO,FLUMINENSE,BAHIA,FLUMINENSE,MARACANA,2.0,0.0,RJ,BA,RJ,3,0,2.0
1,2,1,2000-07-29,16h00,SABADO,VASCO,SPORT,SPORT,SAO JANUARIO,0.0,2.0,RJ,PE,PE,0,3,2.0
2,3,1,2000-07-29,16h00,SABADO,VITORIA,PALMEIRAS,VITORIA,BARRADAO,4.0,1.0,ES,SP,ES,3,0,3.0
3,4,1,2000-07-30,17h00,DOMINGO,BOTAFOGO-RJ,ATLETICO-MG,-,CAIO MARTINS,0.0,0.0,RJ,MG,-,1,1,0.0
4,5,1,2000-07-30,18h30,DOMINGO,JUVENTUDE,FLAMENGO,-,ALFREDO JACONI,1.0,1.0,RS,RJ,-,1,1,0.0


## Tratando dados inválidos

In [25]:
from unicodedata import normalize

def trataString(value):
    return normalize('NFKD', str(value)).encode('ASCII', 'ignore').decode('ASCII').upper()

df['Mandante'] = df['Mandante'].apply(trataString)
df['Visitante'] = df['Visitante'].apply(trataString)
df['Dia'] = df['Dia'].apply(trataString)
df['Vencedor'] = df['Vencedor'].apply(trataString)
df['Arena'] = df['Arena'].apply(trataString)


## Ordenando o Dataset pelo Mandante da partida

In [40]:
df['Pontos Mandante'] = 0
df['Pontos Visitante'] = 0

df.loc[ df['Vencedor'] == df['Mandante'], 'Pontos Mandante' ] = 3
df.loc[ df['Vencedor'] == df['Visitante'], 'Pontos Visitante' ] = 3
df.loc[ df['Vencedor'] == '-', 'Pontos Mandante' ] = 1
df.loc[ df['Vencedor'] == '-', 'Pontos Visitante' ] = 1


times = pd.Series(df['Mandante'].unique(), name='Times')
tg = pd.DataFrame(times)
tg['Pontos'] = 0

test = df.groupby('Mandante')['Pontos Mandante'].sum()
nova = pd.DataFrame(test)

for index, i in nova.iterrows():
    tg.loc[ tg['Times'] == index, 'Pontos' ] = i['Pontos Mandante']

tg = tg.sort_values(by="Pontos", ascending=False).reset_index()
tg[['Times', 'Pontos']]

,Times,Pontos
0,SANTOS,800
1,SAO PAULO,799
2,GREMIO,759
3,ATHLETICO-PR,756
4,INTERNACIONAL,742
5,FLAMENGO,725
6,ATLETICO-MG,711
7,CRUZEIRO,706
8,FLUMINENSE,705
9,CORINTHIANS,701


## Ordenando o Dataset pelas maiores goleadas

In [27]:
df['Diferenca Gols'] = abs(df['Mandante Placar'] - df['Visitante Placar'])

# dropando valores nulos, visto que se não houve o jogo, provavelmente foi remarcado e está em outro registro
df = df.dropna(subset=['Diferenca Gols'])

df.sort_values(by=['Diferenca Gols'], ascending=False)

,ID,Rodada,Data,Horário,Dia,Mandante,Visitante,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor,Pontos Mandante,Pontos Visitante,Diferenca Gols
1600,1601,46,2003-12-14,16h00,DOMINGO,BAHIA,CRUZEIRO,CRUZEIRO,FONTE NOVA,0.0,7.0,BA,MG,MG,0,3,7.0
1122,1123,6,2003-04-27,16h00,DOMINGO,GOIAS,JUVENTUDE,GOIAS,SERRA DOURADA,7.0,0.0,GO,RS,GO,3,0,7.0
1992,1993,33,2004-09-28,20h30,TERCA-FEIRA,SAO PAULO,PAYSANDU,SAO PAULO,MORUMBI,7.0,0.0,SP,PA,SP,3,0,7.0
1528,1529,40,2003-11-02,16h00,DOMINGO,FIGUEIRENSE,PAYSANDU,FIGUEIRENSE,ORLANDO SCARPELLI,6.0,0.0,SC,PA,SC,3,0,6.0
1338,1339,24,2003-08-07,20h30,QUINTA-FEIRA,FLAMENGO,BAHIA,FLAMENGO,MARACANA,6.0,0.0,RJ,BA,RJ,3,0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2740,2741,13,2006-07-22,18h10,SABADO,FLUMINENSE,SAO CAETANO,-,MARACANA,2.0,2.0,RJ,SP,-,1,1,0.0
3750,3751,38,2008-12-07,17h00,DOMINGO,SANTOS,NAUTICO,-,VILA BELMIRO,0.0,0.0,SP,PE,-,1,1,0.0
1666,1667,6,2004-05-16,16h00,DOMINGO,JUVENTUDE,ATLETICO-MG,-,ALFREDO JACONI,1.0,1.0,RS,MG,-,1,1,0.0
6932,6933,14,2017-07-16,16:00,DOMINGO,VASCO,SANTOS,-,ENGENHAO,0.0,0.0,RJ,SP,-,1,1,0.0


## Exibir as 5 maiores contagens de valores únicos de uma única coluna
#### No caso: Quem são os melhores Mandantes ?

In [28]:
df[ df['Mandante'] == df['Vencedor'] ][['Mandante']].value_counts().head()

Mandante     
SANTOS           233
SAO PAULO        231
GREMIO           223
ATHLETICO-PR     221
INTERNACIONAL    217
dtype: int64

## Exiba, no mínimo, três plots. Se possível, use um para variáveis quantitativa e outro para variáveis de qualitativas

In [29]:
# Implementar

## Implemente pelo menos um filtro

In [30]:
# Implementar

## Utilize, no mínimo, uma função agregada com agrupamento

In [31]:
df.groupby(by=['Estado Mandante'])['Mandante Placar'].sum() + df.groupby(by=['Estado Visitante'])['Visitante Placar'].sum()

Estado Mandante
AL      24.0
BA     521.0
CE     477.0
DF     141.0
ES     615.0
GO    1061.0
MG    2438.0
MT      12.0
PA     229.0
PE     880.0
PR    2147.0
RJ    3969.0
RN      24.0
RS    2426.0
SC    1269.0
SP    6011.0
dtype: float64

In [55]:
import datetime

df['Data2'] = pd.to_datetime(df['Data'])
df['Ano'] = df['Data2'].dt.year
df.drop(columns='Data2', inplace=True)

test = df.groupby(['Mandante', 'Ano'])['Pontos Mandante'].sum()
pontuacaoPorAno = pd.DataFrame(test)
pontuacaoPorAno

Pontos Mandante
Mandante   Ano                  
AMERICA-MG 2000               20
           2001               14
           2011               28
           2016               24
           2018               31
...                          ...
VITORIA    2013               39
           2014               23
           2016               28
           2017               14
           2018               26

[477 rows x 1 columns]